In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [10]:
test['Survived'] = 0
train_test = train.append(test)

train_test = pd.get_dummies(train_test,columns=['Pclass'])

train_test = pd.get_dummies(train_test,columns=["Sex"])

train_test['SibSp_Parch'] = train_test['SibSp'] + train_test['Parch']
train_test = pd.get_dummies(train_test,columns = ['SibSp','Parch','SibSp_Parch']) 

train_test = pd.get_dummies(train_test,columns=["Embarked"])


train_test['Name1'] = train_test['Name'].str.extract('.+,(.+)', expand=False).str.extract('^(.+?)\.', expand=False).str.strip()
train_test['Name1'].replace(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer' , inplace = True)
train_test['Name1'].replace(['Jonkheer', 'Don', 'Sir', 'the Countess', 'Dona', 'Lady'], 'Royalty' , inplace = True)
train_test['Name1'].replace(['Mme', 'Ms', 'Mrs'], 'Mrs')
train_test['Name1'].replace(['Mlle', 'Miss'], 'Miss')
train_test['Name1'].replace(['Mr'], 'Mr' , inplace = True)
train_test['Name1'].replace(['Master'], 'Master' , inplace = True)
train_test = pd.get_dummies(train_test,columns=['Name1'])

train_test['Name2'] = train_test['Name'].apply(lambda x: x.split('.')[1])
Name2_sum = train_test['Name2'].value_counts().reset_index()
Name2_sum.columns=['Name2','Name2_sum']
train_test = pd.merge(train_test,Name2_sum,how='left',on='Name2')
train_test.loc[train_test['Name2_sum'] == 1 , 'Name2_new'] = 'one'
train_test.loc[train_test['Name2_sum'] > 1 , 'Name2_new'] = train_test['Name2']
del train_test['Name2']
train_test = pd.get_dummies(train_test,columns=['Name2_new'])

del train_test['Name']

/opt/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [11]:
train_test.loc[train_test["Fare"].isnull()]
train.groupby(by=["Pclass","Embarked"]).Fare.mean()
train_test["Fare"].fillna(14.644083,inplace=True)

train_test['Ticket_Letter'] = train_test['Ticket'].str.split().str[0]
train_test['Ticket_Letter'] = train_test['Ticket_Letter'].apply(lambda x:np.nan if x.isnumeric() else x)
train_test.drop('Ticket',inplace=True,axis=1)
train_test = pd.get_dummies(train_test,columns=['Ticket_Letter'],drop_first=True)

train_test.loc[train_test["Age"].isnull()]['Survived'].mean()
train_test.loc[train_test["Age"].isnull() ,"age_nan"] = 1
train_test.loc[train_test["Age"].notnull() ,"age_nan"] = 0
train_test = pd.get_dummies(train_test,columns=['age_nan'])


missing_age = train_test.drop(['Survived','Cabin'],axis=1)
missing_age_train = missing_age[missing_age['Age'].notnull()]
missing_age_test = missing_age[missing_age['Age'].isnull()]


missing_age_X_train = missing_age_train.drop(['Age'], axis=1)
missing_age_Y_train = missing_age_train['Age']
missing_age_X_test = missing_age_test.drop(['Age'], axis=1)




from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(missing_age_X_train)
missing_age_X_train = ss.transform(missing_age_X_train)
missing_age_X_test = ss.transform(missing_age_X_test)

from sklearn import linear_model
lin = linear_model.BayesianRidge()

lin.fit(missing_age_X_train,missing_age_Y_train)

BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=False)

In [12]:
train_test.loc[(train_test['Age'].isnull()), 'Age'] = lin.predict(missing_age_X_test)

train_test['Age'] = pd.cut(train_test['Age'], bins=[0,10,18,30,50,100],labels=[1,2,3,4,5])

train_test = pd.get_dummies(train_test,columns=['Age'])

train_test['Cabin_nan'] = train_test['Cabin'].apply(lambda x:str(x)[0] if pd.notnull(x) else x)
train_test = pd.get_dummies(train_test,columns=['Cabin_nan'])

train_test.loc[train_test["Cabin"].isnull() ,"Cabin_nan"] = 1
train_test.loc[train_test["Cabin"].notnull() ,"Cabin_nan"] = 0
train_test = pd.get_dummies(train_test,columns=['Cabin_nan'])
train_test.drop('Cabin',axis=1,inplace=True)

In [13]:
train_data = train_test[:891]
test_data = train_test[891:]
train_data_X = train_data.drop(['Survived'],axis=1)
train_data_Y = train_data['Survived']
test_data_X = test_data.drop(['Survived'],axis=1)

In [14]:
from sklearn.preprocessing import StandardScaler
ss2 = StandardScaler()
ss2.fit(train_data_X)
train_data_X_sd = ss2.transform(train_data_X)
test_data_X_sd = ss2.transform(test_data_X)

In [9]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.grid_search import GridSearchCV

# lr = LogisticRegression()
# param = {'C':[0.001,0.01,0.1,1,10], "max_iter":[100,250]}
# clf = GridSearchCV(lr, param,cv=5, n_jobs=-1, verbose=1, scoring="roc_auc")
# clf.fit(train_data_X_sd, train_data_Y)    #????

# # 打印参数的得分情况
# clf.grid_scores_
# # 打印最佳参数
# clf.best_params_

# # 将最佳参数传入训练模型
# lr = LogisticRegression(clf.best_params_)
# lr.fit(train_data_X_sd, train_data_Y)

# # 输出结果
# test["Survived"] = lr.predict(test_data_X_sd)
# test[['PassengerId', 'Survived']].set_index('PassengerId').to_csv('LS5.csv')


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Done  19 out of  50 | elapsed:    0.3s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    1.5s finished


TypeError: unhashable type: 'dict'

In [10]:
train_data_X_sd.info()

AttributeError: 'numpy.ndarray' object has no attribute 'info'

In [14]:
train_data_X_sd

array([[-0.50244517, -1.73010796, -0.56568542, ..., -0.03352008,
        -0.54492498,  0.54492498],
       [ 0.78684529, -1.72622007,  1.76776695, ..., -0.03352008,
         1.835115  , -1.835115  ],
       [-0.48885426, -1.72233219, -0.56568542, ..., -0.03352008,
        -0.54492498,  0.54492498],
       ...,
       [-0.17626324,  1.72233219, -0.56568542, ..., -0.03352008,
        -0.54492498,  0.54492498],
       [-0.04438104,  1.72622007,  1.76776695, ..., -0.03352008,
         1.835115  , -1.835115  ],
       [-0.49237783,  1.73010796, -0.56568542, ..., -0.03352008,
        -0.54492498,  0.54492498]])

In [9]:
train_data.head()

,Fare,PassengerId,Survived,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,SibSp_0,SibSp_1,...,Cabin_nan_A,Cabin_nan_B,Cabin_nan_C,Cabin_nan_D,Cabin_nan_E,Cabin_nan_F,Cabin_nan_G,Cabin_nan_T,Cabin_nan_0.0,Cabin_nan_1.0
0,7.2500,1,0,0,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
1,71.2833,2,1,1,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,1,0
2,7.9250,3,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,53.1000,4,1,1,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,1,0
4,8.0500,5,0,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
train_data_X.head()

,Fare,PassengerId,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,SibSp_0,SibSp_1,SibSp_2,...,Cabin_nan_A,Cabin_nan_B,Cabin_nan_C,Cabin_nan_D,Cabin_nan_E,Cabin_nan_F,Cabin_nan_G,Cabin_nan_T,Cabin_nan_0.0,Cabin_nan_1.0
0,7.2500,1,0,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,71.2833,2,1,0,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
2,7.9250,3,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,53.1000,4,1,0,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
4,8.0500,5,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [14]:
train_data_X_sd[0][0]

-0.5024451714361923

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV

lr = LogisticRegression()
param = {'C':[0.001,0.01,0.1,1,10], "max_iter":[100,250]}
clf = GridSearchCV(lr, param,cv=5, n_jobs=-1, verbose=1, scoring="roc_auc")
clf.fit(train_data_X_sd, train_data_Y)    #????


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Done  19 out of  50 | elapsed:    0.3s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    1.6s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10], 'max_iter': [100, 250]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=1)

In [16]:
# 打印参数的得分情况
clf.grid_scores_
# 打印最佳参数
clf.best_params_

{'C': 0.01, 'max_iter': 100}

In [17]:
# 将最佳参数传入训练模型
lr = LogisticRegression(clf.best_params_)

In [27]:
lr = LogisticRegression(C=0.01, max_iter=100)

lr.fit(train_data_X_sd, train_data_Y)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [28]:
test['Survived'] = lr.predict(test_data_X_sd)
test[['PassengerId', 'Survived']].set_index('PassengerId').to_csv('LS5.csv')

In [20]:
from sklearn import svm
svc = svm.SVC()

clf = GridSearchCV(svc, param, cv=5, n_jobs=-1, verbose=1, scoring='roc_auc')
clf.fit(train_data_X_sd, train_data_Y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.6/site-packages/s

/opt/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=250).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=250).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.6/site-packages/s

GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10], 'max_iter': [100, 250]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=1)

In [13]:
clf.grid_scores_

[mean: 0.77972, std: 0.05584, params: {'C': 0.001, 'max_iter': 100},
 mean: 0.82654, std: 0.04642, params: {'C': 0.001, 'max_iter': 250},
 mean: 0.77894, std: 0.06400, params: {'C': 0.01, 'max_iter': 100},
 mean: 0.82884, std: 0.04105, params: {'C': 0.01, 'max_iter': 250},
 mean: 0.79176, std: 0.05139, params: {'C': 0.1, 'max_iter': 100},
 mean: 0.83312, std: 0.04015, params: {'C': 0.1, 'max_iter': 250},
 mean: 0.82136, std: 0.02603, params: {'C': 1, 'max_iter': 100},
 mean: 0.84149, std: 0.02472, params: {'C': 1, 'max_iter': 250},
 mean: 0.79941, std: 0.02746, params: {'C': 10, 'max_iter': 100},
 mean: 0.82893, std: 0.03382, params: {'C': 10, 'max_iter': 250}]

In [14]:
clf.best_params_

{'C': 1, 'max_iter': 250}

In [21]:
svc = svm.SVC(C=1, max_iter=250)

In [16]:
svc.fit(train_data_X_sd, train_data_Y)
svc.predict(test_data_X_sd)

/opt/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=250).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [22]:
test['Survived'] = svc.predict(test_data_X_sd)
SVM = test[['PassengerId', 'Survived']].set_index('PassengerId')
SVM.to_csv('svm1.csv')

NotFittedError: This SVC instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [23]:
from sklearn.ensemble import GradientBoostingClassifier

gbdt = GradientBoostingClassifier(learning_rate=0.7,
                                 max_depth=6,
                                 n_estimators=100,
                                 min_samples_leaf=2)
gbdt.fit(train_data_X, train_data_Y)

test['Survived'] = gbdt.predict(test_data_X)
test[['PassengerId', 'Survived']].set_index('PassengerId').to_csv('gbdt3.csv')

In [25]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=150, min_samples_leaf=3, max_depth=6, oob_score=True)
rf.fit(train_data_X_sd, train_data_Y)

test['Survived'] = rf.predict(test_data_X)
RF = test[['PassengerId', 'Survived']].set_index('PassengerId')
RF.to_csv('RF.csv')

In [31]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(n_estimators=150,
                             min_samples_leaf=3,
                             max_depth=6)
xgb_model.fit(train_data_X, train_data_Y)

test['Survived'] = xgb_model.predict(test_data_X)
XGB = test[['PassengerId', 'Survived']].set_index('PassengerId')
XGB.to_csv('XGB5.csv')

ImportError: cannot import name 'XGBClassifier'

In [25]:
from xgboost.sklearn import XGBModel

XGBoostLibraryNotFound: Cannot find XGBoost Library in the candidate path, did you install compilers and run build.sh in root path?
List of candidates:
/opt/anaconda3/lib/python3.6/site-packages/xgboost/libxgboost.so
/opt/anaconda3/lib/python3.6/site-packages/xgboost/../../lib/libxgboost.so
/opt/anaconda3/lib/python3.6/site-packages/xgboost/./lib/libxgboost.so
/opt/anaconda3/xgboost/libxgboost.so

In [26]:
import xgboost as xgb

XGBoostLibraryNotFound: Cannot find XGBoost Library in the candidate path, did you install compilers and run build.sh in root path?
List of candidates:
/opt/anaconda3/lib/python3.6/site-packages/xgboost/libxgboost.so
/opt/anaconda3/lib/python3.6/site-packages/xgboost/../../lib/libxgboost.so
/opt/anaconda3/lib/python3.6/site-packages/xgboost/./lib/libxgboost.so
/opt/anaconda3/xgboost/libxgboost.so

In [5]:
from xgboost import XGBClassifier

XGBoostLibraryNotFound: Cannot find XGBoost Library in the candidate path, did you install compilers and run build.sh in root path?
List of candidates:
/opt/anaconda3/lib/python3.6/site-packages/xgboost/libxgboost.so
/opt/anaconda3/lib/python3.6/site-packages/xgboost/../../lib/libxgboost.so
/opt/anaconda3/lib/python3.6/site-packages/xgboost/./lib/libxgboost.so
/opt/anaconda3/xgboost/libxgboost.so

In [30]:
from sklearn.ensemble import VotingClassifier

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=0.1,
                        max_iter=100)

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200,
                           min_samples_leaf=2,
                           max_depth=6,
                           oob_score=True)

from sklearn.ensemble import GradientBoostingClassifier
gbdt = GradientBoostingClassifier(learning_rate=0.1,
                                 min_samples_leaf=2,
                                 max_depth=6,
                                 n_estimators=100)

vot = VotingClassifier(estimators=[('lr',lr), ('rf',rf), ('gbdt',gbdt)], voting='hard')
vot.fit(train_data_X_sd, train_data_Y)

test['Survived'] = vot.predict(test_data_X_sd)
test[['PassengerId', 'Survived']].set_index('PassengerId').to_csv('vot5.csv')

/opt/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [31]:
X = train_data_X_sd
X_predict = test_data_X_sd
y = train_data_Y



In [32]:
clfs = [LogisticRegression(C=0.1,
                          max_iter=100),
       RandomForestClassifier(n_estimators=100,
                             max_depth=6,
                             oob_score=True),
       GradientBoostingClassifier(learning_rate=0.3,
                                 max_depth=6,
                                 n_estimators=100)]

In [33]:
from sklearn.cross_validation import StratifiedKFold

n_folds = 5
skf = list(StratifiedKFold(y, n_folds))

dataset_blend_train = np.zeros((X.shape[0], len(clfs)))
dataset_blend_test = np.zeros((X_predict.shape[0], len(clfs)))

for j, clf in enumerate(clfs):
    dataset_blend_test_j = np.zeros((X_predict.shape[0], len(skf)))
    for i, (train, test) in enumerate(skf):
        X_train, y_train, X_test, y_test = X[train], y[train], X[test], y[test]
        clf.fit(X_train, y_train)
        y_submission = clf.predict_proba(X_test)[:,1]
        dataset_blend_train[test, j] = y_submission
        dataset_blend_test_j[:,i] = clf.predict_proba(X_predict)[:,1]
    dataset_blend_test[:,j] = dataset_blend_test_j.mean(1)
    
clf2 = LogisticRegression(C=0.1,
                         max_iter=100)
clf2.fit(dataset_blend_train, y)
y_submission = clf2.predict_proba(dataset_blend_test[:,1])

test = pd.read_csv('test.csv')
test['Survived'] = clf2.predict(dataset_blend_test)
test[['PassengerId', 'Survived']].set_index('PassengerId').to_csv('stack3.csv')

ValueError: Expected 2D array, got 1D array instead:
array=[0.11794905 0.51595624 0.15389729 0.1312495  0.52280774 0.15013079
 0.55493412 0.26124446 0.59081131 0.1757235  0.13706435 0.24693341
 0.87724481 0.17747811 0.85715019 0.73865469 0.15619945 0.15912483
 0.47000802 0.56910386 0.28229956 0.4194416  0.72134404 0.33595715
 0.88337941 0.18089699 0.8814411  0.16732029 0.3369517  0.21565783
 0.18114175 0.20737268 0.55864944 0.45327778 0.45196599 0.16732029
 0.49190445 0.49238506 0.13703848 0.21203106 0.19624351 0.39184877
 0.14850752 0.69043974 0.8757159  0.17884382 0.40922011 0.11599956
 0.86619241 0.55693376 0.39822909 0.19070358 0.69384583 0.81594844
 0.19973926 0.22791178 0.14953259 0.2331859  0.15027587 0.89718728
 0.14828496 0.17698509 0.1491039  0.54837739 0.6698853  0.70194534
 0.5593981  0.26142578 0.41320948 0.76018997 0.55848148 0.1459516
 0.4972615  0.44956461 0.86865869 0.44358495 0.12092503 0.81734741
 0.16922213 0.55744998 0.55846671 0.39716748 0.24693341 0.13706435
 0.1

In [34]:
clfs = [LogisticRegression(C=0.1,max_iter=100),
        RandomForestClassifier(n_estimators=100,max_depth=6,oob_score=True),
        GradientBoostingClassifier(learning_rate=0.3,max_depth=6,n_estimators=100)]

# 创建n_folds
from sklearn.cross_validation import StratifiedKFold
n_folds = 5
skf = list(StratifiedKFold(y, n_folds))

# 创建零矩阵
dataset_blend_train = np.zeros((X.shape[0], len(clfs)))
dataset_blend_test = np.zeros((X_predict.shape[0], len(clfs)))

# 建立模型
for j, clf in enumerate(clfs):
    '''依次训练各个单模型'''
    # print(j, clf)
    dataset_blend_test_j = np.zeros((X_predict.shape[0], len(skf)))
    for i, (train, test) in enumerate(skf):
        '''使用第i个部分作为预测，剩余的部分来训练模型，获得其预测的输出作为第i部分的新特征。'''
        # print("Fold", i)
        X_train, y_train, X_test, y_test = X[train], y[train], X[test], y[test]
        clf.fit(X_train, y_train)
        y_submission = clf.predict_proba(X_test)[:, 1]
        dataset_blend_train[test, j] = y_submission
        dataset_blend_test_j[:, i] = clf.predict_proba(X_predict)[:, 1]
    '''对于测试集，直接用这k个模型的预测值均值作为新的特征。'''
    dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)

# 用建立第二层模型
clf2 = LogisticRegression(C=0.1,max_iter=100)
clf2.fit(dataset_blend_train, y)
y_submission = clf2.predict_proba(dataset_blend_test)[:, 1]

test = pd.read_csv("test.csv")
test["Survived"] = clf2.predict(dataset_blend_test)
test[['PassengerId','Survived']].set_index('PassengerId').to_csv('stack3.csv')